#AUTENTICACIÓN CON GOOGLE Y CONEXIÓN A SHEET

In [24]:
from google.colab import auth

import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth import default

# Autorización
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# URL de la hoja
sheet_url = "https://docs.google.com/spreadsheets/d/1rZdcjfJtz0Bg40XAYb8_6Z70Sf_fMrMb6LXeijdlodk/edit?gid=0#gid=0"
sh = gc.open_by_url(sheet_url)
worksheet = sh.sheet1

df = get_as_dataframe(worksheet, evaluate_formulas=True)
df = df.dropna(how="all")

#MODELO V1: Árbol de decisión para predecir salario según tecnologías


In [10]:
import pandas as pd
import numpy as np
import re
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score

# --- 🔹 1️⃣ Cargar los datos (ajusta el nombre del archivo si es necesario) ---
# Debe contener: Titulo, Descripción breve, Ciudad, Salario, Modalidad, Pagina (link)

df = get_as_dataframe(worksheet, evaluate_formulas=True).dropna(how="all")

# --- 🔹 2️⃣ Definir lista de tecnologías válidas ---
tecnologias_validas = [
    "python", "java", "javascript", "typescript", "angular", "react", "vue",
    "node", "nodejs", "php", "laravel", "django", "flask", "spring", "kotlin",
    "swift", "flutter", "docker", "kubernetes", "git", "html", "css", "sql",
    "mysql", "postgresql", "mongodb", "firebase", "aws", "azure", "gcp",
    "linux", "bash", "powershell", "graphql", "nextjs", "nuxt", "tailwind",
    "reactnative", "redux", "express", "rest", "api", "go", "golang", "c#",
    "c++", "unity", "unreal", "pandas", "numpy", "tensorflow", "pytorch"
]

# --- 🔹 3️⃣ Limpieza de salarios ---
def limpiar_salario(s):
    if pd.isna(s):
        return np.nan

    s = str(s).lower().strip()
    s = re.sub(r"(mensual|cop|col|pesos|aprox|al mes|por mes|por hora)", "", s)

    if "millon" in s or "millones" in s:
        numeros = re.findall(r"[\d.,]+", s)
        if numeros:
            valor = float(numeros[0].replace(",", "."))
            return int(valor * 1_000_000)

    s = s.replace("$", "").replace(" ", "")
    s = re.sub(r"(?<=\d)[.,](?=\d{3}\b)", "", s)
    numeros = re.findall(r"\d+", s)
    if not numeros:
        return np.nan

    valor = int(numeros[0])
    if valor < 500_000:
        valor *= 1_000
    elif valor > 100_000_000:
        valor = np.nan
    return valor

df["Salario_num"] = df["Salario"].apply(limpiar_salario)
df = df.dropna(subset=["Salario_num"])
df = df[(df["Salario_num"] > 800000) & (df["Salario_num"] < 30000000)]

# --- 🔹 4️⃣ Extraer tecnologías de cada oferta ---
def extraer_tecnologias(texto, lista_validas):
    texto = str(texto).lower()
    palabras = re.findall(r'\b[a-zA-Z0-9\+\#]+\b', texto)
    return [p for p in palabras if p in lista_validas]

df["tecnologias_detectadas"] = df["Descripción breve"].apply(
    lambda x: extraer_tecnologias(x, tecnologias_validas)
)

# --- 🔹 5️⃣ Crear columnas binarias (una por tecnología) ---
for tech in tecnologias_validas:
    df[tech] = df["tecnologias_detectadas"].apply(lambda x: 1 if tech in x else 0)

# --- 🔹 6️⃣ Entrenar el modelo ---
X = df[tecnologias_validas]
y = df["Salario_num"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = DecisionTreeRegressor(max_depth=10, random_state=42) #Niveles
modelo.fit(X_train, y_train)

# --- 🔹 7️⃣ Evaluar ---
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"📊 Error medio absoluto (MAE): {mae:,.0f} COP")
print(f"📈 Coeficiente R²: {r2:.2f}")

# --- 🔹 8️⃣ Mostrar reglas del árbol ---
print("\n📜 Reglas del árbol de decisión:")
print(export_text(modelo, feature_names=tecnologias_validas))

# --- 🔹 9️⃣ Función para predecir salario según tecnologías ---
def predecir_salario(lista_tecnologias):
    entrada = pd.DataFrame([{
        tech: 1 if tech in lista_tecnologias else 0
        for tech in tecnologias_validas
    }])
    prediccion = modelo.predict(entrada)[0]
    print(f"\n💻 Tecnologías: {', '.join(lista_tecnologias)}")
    print(f"💰 Salario estimado: ${prediccion:,.0f} COP")
    return prediccion

# --- 🔹 🔟 Ejemplos ---
predecir_salario(["angular", "api", "node","python"])
predecir_salario(["php", "laravel", "mysql"])
predecir_salario(["java", "spring", "docker"])


# --- 4️⃣ Dividir los datos ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("✅ Datos cargados correctamente.")
print(f"📊 Total de muestras: {len(df)}")
print(f"🧩 Características de entrada: {list(X.columns)}")

# --- 5️⃣ Entrenar el árbol de decisión ---
tree = DecisionTreeClassifier(max_depth=4, random_state=42)
tree.fit(X_train, y_train)

print("\n🌳 Entrenamiento completado.")
print(f"📈 Precisión en entrenamiento: {tree.score(X_train, y_train):.2f}")
print(f"📊 Precisión en prueba: {tree.score(X_test, y_test):.2f}")

# --- 6️⃣ Mostrar estructura del árbol ---
print("\n🧠 Árbol de decisión generado:")
print(export_text(tree, feature_names=list(X.columns)))

# --- 7️⃣ Evaluar ---
y_pred = tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ Exactitud final del modelo: {accuracy:.2f}")


📊 Error medio absoluto (MAE): 2,071,566 COP
📈 Coeficiente R²: -0.10

📜 Reglas del árbol de decisión:
|--- kubernetes <= 0.50
|   |--- javascript <= 0.50
|   |   |--- java <= 0.50
|   |   |   |--- react <= 0.50
|   |   |   |   |--- mysql <= 0.50
|   |   |   |   |   |--- linux <= 0.50
|   |   |   |   |   |   |--- python <= 0.50
|   |   |   |   |   |   |   |--- php <= 0.50
|   |   |   |   |   |   |   |   |--- sql <= 0.50
|   |   |   |   |   |   |   |   |   |--- git <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- value: [3883248.64]
|   |   |   |   |   |   |   |   |   |--- git >  0.50
|   |   |   |   |   |   |   |   |   |   |--- value: [3000001.00]
|   |   |   |   |   |   |   |   |--- sql >  0.50
|   |   |   |   |   |   |   |   |   |--- aws <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- value: [1500001.00]
|   |   |   |   |   |   |   |   |   |--- aws >  0.50
|   |   |   |   |   |   |   |   |   |   |--- value: [4000001.00]
|   |   |   |   |   |   |   |--- php >  0.50
|   |   |   

#MODELO V2: Árbol de decisión para predecir salario según tecnologías, ciudad y modalidad

In [15]:
import pandas as pd
import numpy as np
import re
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# --- Cargar los datos ---
# Debe contener: Titulo, Descripción breve, Ciudad, Salario, Modalidad, Pagina (link)
df = get_as_dataframe(worksheet, evaluate_formulas=True).dropna(how="all")

# --- Lista de tecnologías ---
tecnologias_validas = [
    "python", "java", "javascript", "typescript", "angular", "react", "vue",
    "node", "nodejs", "php", "laravel", "django", "flask", "spring", "kotlin",
    "swift", "flutter", "docker", "kubernetes", "git", "html", "css", "sql",
    "mysql", "postgresql", "mongodb", "firebase", "aws", "azure", "gcp",
    "linux", "bash", "powershell", "graphql", "nextjs", "nuxt", "tailwind",
    "reactnative", "redux", "express", "rest", "api", "go", "golang", "c#",
    "c++", "unity", "unreal", "pandas", "numpy", "tensorflow", "pytorch"
]

# --- Limpieza de salarios ---
def limpiar_salario(s):
    if pd.isna(s):
        return np.nan
    s = str(s).lower().strip()
    s = re.sub(r"(mensual|cop|col|pesos|aprox|al mes|por mes|por hora)", "", s)
    if "millon" in s or "millones" in s:
        numeros = re.findall(r"[\d.,]+", s)
        if numeros:
            valor = float(numeros[0].replace(",", "."))
            return int(valor * 1_000_000)
    s = s.replace("$", "").replace(" ", "")
    s = re.sub(r"(?<=\d)[.,](?=\d{3}\b)", "", s)
    numeros = re.findall(r"\d+", s)
    if not numeros:
        return np.nan
    valor = int(numeros[0])
    if valor < 500_000:
        valor *= 1_000
    elif valor > 100_000_000:
        valor = np.nan
    return valor

df["Salario_num"] = df["Salario"].apply(limpiar_salario)
df = df.dropna(subset=["Salario_num"])
df = df[(df["Salario_num"] > 800000) & (df["Salario_num"] < 30000000)]

# --- Extraer tecnologías ---
def extraer_tecnologias(texto, lista_validas):
    texto = str(texto).lower()
    palabras = re.findall(r'\b[a-zA-Z0-9\+\#]+\b', texto)
    return [p for p in palabras if p in lista_validas]

df["tecnologias_detectadas"] = df["Descripción breve"].apply(
    lambda x: extraer_tecnologias(x, tecnologias_validas)
)

# --- Crear columnas binarias por tecnología ---
for tech in tecnologias_validas:
    df[tech] = df["tecnologias_detectadas"].apply(lambda x: 1 if tech in x else 0)

# --- Variables categóricas (Ciudad y Modalidad) ---
df["Ciudad"] = df["Ciudad"].fillna("Desconocida")
df["Modalidad"] = df["Modalidad"].fillna("No especificada")

# Convertir categorías a variables dummy (0/1)
dummies_ciudad = pd.get_dummies(df["Ciudad"], prefix="Ciudad")
dummies_modalidad = pd.get_dummies(df["Modalidad"], prefix="Modalidad")

# Unir todo al dataframe
df = pd.concat([df, dummies_ciudad, dummies_modalidad], axis=1)

# --- Definir variables de entrada y salida ---
X = df[tecnologias_validas + list(dummies_ciudad.columns) + list(dummies_modalidad.columns)]
y = df["Salario_num"]

# --- Entrenar el modelo ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = DecisionTreeRegressor(
    max_depth=15,         # Más niveles → más detalle, pero a demaciada profundidad aprende demaciado de los datos de entrenamiento y llega a un sobre ajuste
    min_samples_split=5,  # Evita dividir nodos con pocos datos
    min_samples_leaf=3,   # Evita hojas muy pequeñas
    random_state=42
)
modelo.fit(X_train, y_train)

# --- Evaluación ---
y_pred = modelo.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"📊 Error medio absoluto (MAE): {mae:,.0f} COP")
print(f"📈 Coeficiente R²: {r2:.2f}")

# --- 🔹 🔟 Reglas del árbol ---
print("\n📜 Reglas del árbol de decisión:")
print(export_text(modelo, feature_names=list(X.columns)))

# --- Función para predecir ---
def predecir_salario(lista_tecnologias, ciudad="Desconocida", modalidad="No especificada"):
    entrada = pd.DataFrame([{
        **{tech: 1 if tech in lista_tecnologias else 0 for tech in tecnologias_validas},
        **{col: 0 for col in dummies_ciudad.columns},
        **{col: 0 for col in dummies_modalidad.columns}
    }])
    # Activar la ciudad y modalidad si existen
    if f"Ciudad_{ciudad}" in entrada.columns:
        entrada[f"Ciudad_{ciudad}"] = 1
    if f"Modalidad_{modalidad}" in entrada.columns:
        entrada[f"Modalidad_{modalidad}"] = 1

    prediccion = modelo.predict(entrada)[0]
    print(f"\n💻 Tecnologías: {', '.join(lista_tecnologias)}")
    print(f"📍 Ciudad: {ciudad} | 🏠 Modalidad: {modalidad}")
    print(f"💰 Salario estimado: ${prediccion:,.0f} COP")
    return prediccion

# --- 🔹 Ejemplos ---
predecir_salario(["angular", "api", "node","python"], ciudad="Bogotá", modalidad="Remoto")
predecir_salario(["php", "laravel", "mysql"], ciudad="Medellín", modalidad="Presencial")
predecir_salario(["java", "spring", "docker"], ciudad="Cali", modalidad="Híbrido")

print("\n✅ Modelo completado con Ciudad y Modalidad incluidas.")


📊 Error medio absoluto (MAE): 1,541,475 COP
📈 Coeficiente R²: 0.20

📜 Reglas del árbol de decisión:
|--- Modalidad_No especificada <= 0.50
|   |--- Ciudad_Medellín, Antioquia <= 0.50
|   |   |--- Modalidad_Remoto <= 0.50
|   |   |   |--- Ciudad_Estados Unidos <= 0.50
|   |   |   |   |--- Modalidad_ Presencial y remoto <= 0.50
|   |   |   |   |   |--- Ciudad_Pereira, Risaralda <= 0.50
|   |   |   |   |   |   |--- Modalidad_ Empleo destacado <= 0.50
|   |   |   |   |   |   |   |--- Modalidad_ Remoto <= 0.50
|   |   |   |   |   |   |   |   |--- value: [6000000.00]
|   |   |   |   |   |   |   |--- Modalidad_ Remoto >  0.50
|   |   |   |   |   |   |   |   |--- Ciudad_Bogotá, D.C., Bogotá, D.C. <= 0.50
|   |   |   |   |   |   |   |   |   |--- value: [2233333.33]
|   |   |   |   |   |   |   |   |--- Ciudad_Bogotá, D.C., Bogotá, D.C. >  0.50
|   |   |   |   |   |   |   |   |   |--- value: [4657416.00]
|   |   |   |   |   |   |--- Modalidad_ Empleo destacado >  0.50
|   |   |   |   |   |   |   

#RANDOM FOREST ROBUSTO PARA PREDECIR SALARIO
##Multiples arboles de desición
Tiempo de ejecucion promedio 1 minuto con 23 segundos

In [23]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ===========================================
# Cargar los datos
# ===========================================

df = get_as_dataframe(worksheet, evaluate_formulas=True).dropna(how="all")
print("✅ Datos originales cargados:", df.shape)

# Normalizar nombres de columnas
df.columns = df.columns.str.strip().str.lower()
print("🧾 Columnas detectadas:", df.columns.tolist())

# Buscar columnas por nombre aproximado
col_titulo = next((c for c in df.columns if "titul" in c), None)
col_desc = next((c for c in df.columns if "descrip" in c), None)
col_ciudad = next((c for c in df.columns if "ciudad" in c), None)
col_modalidad = next((c for c in df.columns if "modal" in c), None)
col_salario = next((c for c in df.columns if "salar" in c or "sueldo" in c), None)

if not all([col_titulo, col_desc, col_ciudad, col_modalidad, col_salario]):
    raise ValueError("❌ No se detectaron todas las columnas requeridas. Verifica los nombres en Google Sheets.")

# ===========================================
# Limpieza robusta del salario
# ===========================================

def limpiar_salario(s):
    """Convierte expresiones de salario en texto a un valor numérico promedio en COP."""
    if pd.isna(s):
        return np.nan

    s = str(s).lower().strip()

    # Casos que indican que no hay salario numérico
    if any(x in s for x in ["confidencial", "a convenir", "no aplica", "no especifica"]):
        return np.nan

    # Unificar formato decimal
    s = s.replace(",", ".")
    # Eliminar símbolos innecesarios
    s = re.sub(r'(cop|col|pesos?|mensual|al mes|por mes|aprox|m\.?|comisiones?)', '', s)
    s = s.replace("$", "").replace("millón", "millones").strip()

    # Detectar rangos (ej. "1.500.000 - 2.000.000", "8 a 10 millones")
    if "-" in s or " a " in s:
        numeros = re.findall(r'[\d.]+', s)
        if len(numeros) >= 2:
            valores = []
            for n in numeros[:2]:
                # Quitar puntos de miles antes de convertir
                n = n.replace(".", "")
                try:
                    val = float(n)
                except:
                    continue
                # Ajuste de magnitud
                if val < 1000:
                    val *= 1_000_000
                elif val < 50_000:
                    val *= 1_000
                valores.append(val)
            return np.mean(valores) if valores else np.nan

    # Detectar un único valor (ej. "$6.500.000,00", "$4,5 millones")
    numeros = re.findall(r'[\d.]+', s)
    if not numeros:
        return np.nan

    n = numeros[0].replace(".", "")
    try:
        val = float(n)
    except:
        return np.nan

    # Ajuste heurístico según magnitud
    if "millon" in s or "millones" in s:
        val *= 1_000_000
    elif val < 1000:
        val *= 1_000_000
    elif val < 50_000:
        val *= 1_000

    return round(val)


df["salario_num"] = df[col_salario].apply(limpiar_salario)
df = df.dropna(subset=["salario_num"])
df = df[(df["salario_num"] > 800_000) & (df["salario_num"] < 30_000_000)]

print("💰 Después de limpiar salario:", df.shape)

# ===========================================
# Tecnologías y texto
# ===========================================

tecnologias = [
    "python", "java", "javascript", "typescript", "angular", "react", "vue",
    "node", "nodejs", "php", "laravel", "django", "flask", "spring", "kotlin",
    "swift", "flutter", "docker", "kubernetes", "git", "html", "css", "sql",
    "mysql", "postgresql", "mongodb", "firebase", "aws", "azure", "gcp",
    "linux", "bash", "powershell", "graphql", "nextjs", "nuxt", "tailwind",
    "reactnative", "redux", "express", "rest", "api", "go", "golang", "c#",
    "c++", "unity", "unreal", "pandas", "numpy", "tensorflow", "pytorch"
]

df["texto_total"] = df[[col_titulo, col_desc]].astype(str).agg(" ".join, axis=1).str.lower()

for tech in tecnologias:
    df[tech] = df["texto_total"].str.contains(rf"\b{tech}\b", regex=True).astype(int)

# ===========================================
# Ciudades y modalidades
# ===========================================

df[col_ciudad] = df[col_ciudad].astype(str).str.strip()
df[col_modalidad] = df[col_modalidad].astype(str).str.strip()

top_cities = df[col_ciudad].value_counts().head(10).index
df[col_ciudad] = df[col_ciudad].apply(lambda x: x if x in top_cities else "Otras")

top_modalidades = df[col_modalidad].value_counts().head(5).index
df[col_modalidad] = df[col_modalidad].apply(lambda x: x if x in top_modalidades else "Otra")

# ===========================================
# Variables predictoras
# ===========================================

X = df[[col_ciudad, col_modalidad] + tecnologias]
y = df["salario_num"]

print("📈 Datos finales para modelo:", X.shape, y.shape)

if len(df) < 50:
    raise ValueError("❌ Muy pocos datos después de la limpieza. Revisa los formatos del salario o nombres de columnas.")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ===========================================
# Modelo y preprocesamiento
# ===========================================

categorical_features = [col_ciudad, col_modalidad]
numeric_features = tecnologias

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
], remainder="passthrough")

rf = RandomForestRegressor(random_state=42)

pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("regressor", rf)
])

# ===========================================
# Búsqueda de hiperparámetros
# ===========================================

param_grid = {
    "regressor__n_estimators": [200, 300, 500],
    "regressor__max_depth": [10, 15, 20],
    "regressor__min_samples_split": [2, 5, 10],
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="r2",
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# ===========================================
# Evaluación final
# ===========================================

y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring="r2")

print(f"✅ Mejores hiperparámetros: {grid_search.best_params_}")
print(f"📊 Error medio absoluto (MAE): {mae:,.0f} COP")
print(f"📈 Coeficiente R²: {r2:.2f}")
print(f"🔁 R² promedio (validación cruzada): {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")


✅ Datos originales cargados: (1450, 6)
🧾 Columnas detectadas: ['titulo', 'pagina', 'descripción breve', 'ciudad', 'salario', 'modalidad']
💰 Después de limpiar salario: (479, 7)
📈 Datos finales para modelo: (479, 54) (479,)
Fitting 5 folds for each of 27 candidates, totalling 135 fits
✅ Mejores hiperparámetros: {'regressor__max_depth': 10, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 200}
📊 Error medio absoluto (MAE): 3,080,943 COP
📈 Coeficiente R²: 0.58
🔁 R² promedio (validación cruzada): 0.47 ± 0.31
